In [1]:
#reading the necessary libraries: #base model
import scipy.sparse as sp
from scipy.sparse import hstack 
import csv
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score,classification_report

#seeding for randamziation:
import numpy as np
np.random.seed(42)
import random
random.seed(42)

In [2]:
#reading raw data from Final_Annotationsby_Detectives_GSD.csv Golden Standards dataset: 
#Feature and Label Variables:
X_txt= []
y= []

# Loading data from CSVs:
# Load the training datasets into two lists (X_txt will be a list of strings with features; 
# y will list of 0's and 1's with labels or classification):
with open('./Gold Standards DataSet_D_SA.csv',encoding='iso-8859-1') as in_file:
    iCSV = csv.reader(in_file, delimiter=',')
    header=next(iCSV)
    for row in iCSV:
        X_txt.append(row[1])
        y.append([int(value) for value in row[2:6]])
        
#print(len(X_txt))
#print(len(y))
#print(header[1:6],y[0])

In [3]:

#Split the data into training and test sets:
X_train,X_test,y_train,y_test=train_test_split(X_txt,y,test_size=0.2)


In [4]:

#modeling with ngram_range=(1,1) & LogisticRegression with CountVectorizer: 

#converting list to matrix:
vec=CountVectorizer(ngram_range=(1,1))
X_train_matrix =vec.fit_transform(X_train) # This should be a matrix
X_test_matrix=vec.transform(X_test)# This should be a matrix

#vec=CountVectorizer(ngram_range=(1,1))
#X_train_matrix =vec.fit_transform(X_txt) # This should be a matrix
#y_matrix=vec.transform(y)# This should be a matrix

#Split the data into training and test sets:
#X_train,X_test,y_train,y_test=train_test_split(X_train_matrix,y,test_size=0.2)

#converting list to array:
ya_train=np.array(y_train)
ya_test=np.array(y_test)
#print(y_test.shape[1])

#initializing logisticregression:
log_reg=MultiOutputClassifier(LogisticRegression(random_state=42,solver='lbfgs', max_iter=2000))

#params with c values:
params= {"estimator__C": [0.0001, 0.001, 0.01, 0.1, 1, 10,100]}

#initialize GridSearchCV with scoring f1_macro:

init_grid_search_macro=GridSearchCV(log_reg,params,cv=5,scoring='f1_macro')

# Fit the model on X_train with scoring f1_macro:
init_grid_search_macro.fit(X_train_matrix,ya_train)

#Validation Score with scoring f1_macro:
validation_macro_score = init_grid_search_macro.score(X_test_matrix,ya_test) 
validation_results_best_score=init_grid_search_macro.best_score_
# Get the score from the GridSearchCV "best score" with Macro f1:
print("Macro Validation Score F1: {:.4f}".format(validation_results_best_score))
print("Macro Test Score F1: {:.4f}".format(validation_macro_score))

#predciting on X_test data with scoring f1_macro:
logistic_X_test_prediciton_macro=init_grid_search_macro.predict(X_test_matrix)

# Calculating precision, recall and f1 Scores with average as macro parameter:
precision_macro=precision_score(ya_test,logistic_X_test_prediciton_macro,average='macro')  # Get scores using logistic_X_test_prediciton and y_test with the precision_score method
recall_macro = recall_score(ya_test,logistic_X_test_prediciton_macro,average='macro')
print("Macro_Score Precision: {:.4f}".format(precision_macro))
print("Macro_Score Recall: {:.4f}".format(recall_macro))
for i in range(4):
    #f1_macro_i = f1_score([item[i] for item in y_test], [item[i] for item in logistic_X_test_prediciton_macro])
    f1_macro_i = f1_score(ya_test[:,i],logistic_X_test_prediciton_macro[:,i])
    print(f"{header[i+2]} Macro_Score F1: {f1_macro_i:.4f}")
    
print()    
    

#initialize GridSearchCV with scoring f1_micro:

init_grid_search_micro=GridSearchCV(log_reg,params,cv=5,scoring='f1_micro')

# Fit the model on X_train with scoring f1_micro:
init_grid_search_micro.fit(X_train_matrix,ya_train)

#Validation Score with scoring f1_macro:
validation_micro_score = init_grid_search_micro.score(X_test_matrix,ya_test) 
validation_results_best_score_micro=init_grid_search_micro.best_score_
# Get the score from the GridSearchCV "best score" with Macro f1:
print("Micro Validation Score F1: {:.4f}".format(validation_results_best_score_micro))
print("Micro Test Score F1: {:.4f}".format(validation_micro_score))

# Get the score from the GridSearchCV "best score" with Micro f1:
#print("Micro_Score Validation F1: {:.4f}".format(validation_micro_score))

#predciting on X_test data with scoring f1_micro:
logistic_X_test_prediciton_micro=init_grid_search_micro.predict(X_test_matrix)

# Calculating precision, recall and f1 Scores with average as micro parameter:
precision_micro=precision_score(ya_test,logistic_X_test_prediciton_micro,average='micro')  # Get scores using logistic_X_test_prediciton and y_test with the precision_score method
recall_micro = recall_score(ya_test,logistic_X_test_prediciton_micro,average='micro')
print("Micro_Score Precision: {:.4f}".format(precision_micro))
print("Micro_Score Recall: {:.4f}".format(recall_micro))
for i in range(4):
    #f1_micro_i = f1_score([item[i] for item in y_test], [item[i] for item in logistic_X_test_prediciton_micro])
    f1_micro_i = f1_score(ya_test[:,i],logistic_X_test_prediciton_micro[:,i])
    print(f"{header[i+2]} Micro_Score F1: {f1_micro_i:.4f}")    

Macro Validation Score F1: 0.3262
Macro Test Score F1: 0.3120
Macro_Score Precision: 0.3233
Macro_Score Recall: 0.3078
Gold Standards- Technology Macro_Score F1: 0.8201
Gold Standards- Ride Share Macro_Score F1: 0.0909
Gold Standards- Food Delivery Macro_Score F1: 0.1647
Gold Standards- Online Shopping Macro_Score F1: 0.1724

Micro Validation Score F1: 0.5797
Micro Test Score F1: 0.5450
Micro_Score Precision: 0.6124
Micro_Score Recall: 0.4910
Gold Standards- Technology Micro_Score F1: 0.7397
Gold Standards- Ride Share Micro_Score F1: 0.0667
Gold Standards- Food Delivery Micro_Score F1: 0.0000
Gold Standards- Online Shopping Micro_Score F1: 0.0000


In [5]:
class LexiconClassifier():
    def __init__(self):
        self.positive_words = set()
        self.capital_words = set()
        self.exclamation_points = set()
        self.negative_words = set()
        self.online_apps_words=set()
        
    def online_apps(self, file_path):
        with open('./online-words.txt',encoding='iso-8859-1') as iFile:
            for row in iFile:
                self.online_apps_words.add(row.strip())
    def load_capital_words(self, file_path):
        with open('./Gold Standards DataSet_D_SA.csv',encoding='iso-8859-1') as iFile:
            for row in iFile:
                self.capital_words.add(row.strip())

    def load_exclamation_points(self, file_path):
        with open('./Gold Standards DataSet_D_SA.csv',encoding='iso-8859-1') as iFile:
            for row in iFile:
                self.exclamation_points.add(row.strip())
    def load_positive_negative_words(self, file_path):
        with open('./Gold Standards DataSet_D_SA.csv',encoding='iso-8859-1') as iFile:
            for row in iFile:
                self.positive_words.add(row.strip())
                
        with open('./Gold Standards DataSet_D_SA.csv', encoding='iso-8859-1') as iFile:
            for row in iFile:
                self.negative_words.add(row.strip())
                
    def predict_positive_negtaive(self, sentence):
        """
            Returns a sentiment prediction give an input string.
            
            Keyword arguments:
            sentence -- string (e.g., "This is good good good")
            
            Returns:
            pred -- a string ("postive, "negative", or "neutral")
        """
        num_pos_words = 0
        num_neg_words = 0
        for word in sentence.lower().split():
            if word in self.positive_words:
                num_pos_words += 1
            elif word in self.negative_words:
                num_neg_words += 1
        
        pred_pos_neg = 'neutral'        
        if num_pos_words > num_neg_words:
            pred_pos_neg = 'positive'
        elif num_pos_words < num_neg_words:
            pred_pos_neg = 'negative'
            
        return pred_pos_neg
    
    
    def predict_capital_words(self, sentence):
        """
            Returns the number of capital words in a string.

            Keyword arguments:
            sentence -- string (e.g., "This is GOOD good GOOD")

            Returns:
            num_capital_words -- an integer (e.g., 3)
        """
        num_capital_words = 0
        for word in sentence.lower().split():
            if word.isupper() and word in self.capital_words:
                num_capital_words += 1
        return num_capital_words

    def predict_exclamation_points(self, sentence):
        """
            Returns the number of exclamation points in a string.

            Keyword arguments:
            sentence -- string (e.g., "This is great!!!")

            Returns:
            num_exclamation_points -- an integer (e.g., 3)
        """
        num_exclamation_points = 0
        for char in sentence:
            if char == '!':
                num_exclamation_points += 1
        return num_exclamation_points
    
    def count_pos_words(self, sentence):
        """
            Returns the number of positive words in string
            
            Keyword arguments:
            sentence -- string (e.g., "This is good good good")
            
            Returns:
            pred -- an integer (e.g., 3)
        """
        num_pos_words = 0
        for word in sentence.lower().split():
            if word in self.positive_words:
                num_pos_words += 1
        return num_pos_words
    def count_neg_words(self, sentence):
        """
            Returns the number of negative words in string
            
            Keyword arguments:
            sentence -- string (e.g., "This is good good good")
            
            Returns:
            pred -- an integer (e.g., 3)
        """
        num_neg_words = 0
        for word in sentence.lower().split():
            if word in self.negative_words:
                num_neg_words += 1
        return num_neg_words
    
    def predict_online_apps(self, sentence):
            
        """
        Returns True if specific online application words are found in a string.

        Keyword arguments:
        sentence -- string (e.g., "I ordered from ubereats and doordash yesterday.")

        Returns:
        online_apps_present -- a boolean
        """
        online_apps_count = sum(word in sentence.lower() for word in self.online_apps_words)
        return online_apps_count

In [6]:
# WRITE CODE HERE
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# 1. Instatiate that class

lex_test_preds_P_N = [] # Initialize this as an empty list
lex_test_preds_Capital = [] 
lex_test_preds_exclamation=[]
lex_test_preds_positive_wrd=[]
lex_test_preds_negative_wrd=[]
lex_test_preds_online_apps_cnt=[]
# Loop over X_txt_test
#    for each string in X_txt_test (i.e., for each item in the list), pass it to LexiconClassifiers .predict() method
#    append the prediction to lex_test_preds

LexiconClassifier_v=LexiconClassifier()
for i in X_test:
    
    #Predict positive and negative words in the sentence:
    Predict_Positive_lexicon=LexiconClassifier_v.predict_positive_negtaive(i)
    lex_test_preds_P_N.append(Predict_Positive_lexicon)
    #Predict capital words in the senetence:
    Predict_Capital_lexicon=LexiconClassifier_v.predict_capital_words(i)
    lex_test_preds_Capital.append(Predict_Capital_lexicon)
    #Predict exclamation(!) marks in the senetence:
    Predict_exclamation_lexicon=LexiconClassifier_v.predict_exclamation_points(i)
    lex_test_preds_exclamation.append(Predict_exclamation_lexicon)
    #Predict count of positive words in the senetence:
    Predict_positive_lexicon=LexiconClassifier_v.count_pos_words(i)
    lex_test_preds_positive_wrd.append(Predict_positive_lexicon)
    #Predict count of negative words in the senetence:
    Predict_negative_lexicon=LexiconClassifier_v.count_neg_words(i)
    lex_test_preds_negative_wrd.append(Predict_negative_lexicon)
    #Predict count of negative words in the senetence:
    Predict_online_apps_lexicon=LexiconClassifier_v.predict_online_apps(i)
    lex_test_preds_online_apps_cnt.append(Predict_online_apps_lexicon)
#print(lex_test_preds_exclamation[1:100])

In [7]:
# WRITE CODE HERE

X_train_lexicon_features_pn = [] # Initailze to an empty list. This will be a list of li #  Initailze to an empty list. This will be a list of lists
X_test_lexicon_features_pn=[]
X_test_lexicon_features_em=[]
X_train_lexicon_features_em=[]
# Loop over X_txt_test
#    for each string in X_txt_test (i.e., for each item in the list), pass it to LexiconClassifiers .count_pos_words() and count_neg_words method
#    append a list with the counts to X_test_lexicon_features
LexiconClassifier_v=LexiconClassifier()
#count of positive and negative words:
for i in X_test:
    num_pos_test_count=LexiconClassifier_v .count_pos_words(i)
    num_neg_test_count=LexiconClassifier_v .count_neg_words(i)
    X_test_lexicon_features_pn.append([num_pos_test_count,num_neg_test_count])
#print(X_test_lexicon_features)
for j in X_train:
    num_pos_train_count=LexiconClassifier_v .count_pos_words(j)
    num_neg_train_count=LexiconClassifier_v .count_neg_words(j)
    X_train_lexicon_features_pn.append([num_pos_train_count,num_neg_train_count])
    
#count of exclamation points:    
for a in X_test:
    num_exclamation_test_count=LexiconClassifier_v .predict_exclamation_points(a)
    X_test_lexicon_features_em.append(num_exclamation_test_count)
#print(X_test_lexicon_features_em)

for b in X_train:
    num_exclamation_train_count=LexiconClassifier_v .predict_exclamation_points(b)
    X_train_lexicon_features_em.append(num_exclamation_train_count)
#print(X_train_lexicon_features_em)

In [8]:
#modeling with count of exclamation marks ngram_range=(1,1) & LogisticRegression with CountVectorizer: 

#converting list to matrix for exclamation marks:
vec_em=CountVectorizer(ngram_range=(1,1))
X_train_matrix_em =vec_em.fit_transform(X_train) # This should be a matrix
X_test_matrix_em=vec_em.transform(X_test)# This should be a matrix

# Now we need to convert X_train_lexicon_features and X_test_lexicon_features to numpy arrays
# "hstack" X_train_lexicon_features with X_train_w_lex
# "hstack" X_test_lexicon_features with X_test_w_lex
X_train_lexicon_farray_em=np.array(X_train_lexicon_features_em).reshape(-1, 1)
X_test_lexicon_farray_em=np.array(X_test_lexicon_features_em).reshape(-1, 1)

X_train_f_em_lex=hstack([X_train_matrix_em,X_train_lexicon_farray_em])
X_test_f_em_lex=hstack([X_test_matrix_em,X_test_lexicon_farray_em])

#converting list to array:
ya_train_em=np.array(y_train)
ya_test_em=np.array(y_test)
#print(y_test.shape[1])

#initializing logisticregression:
log_reg_f_em=MultiOutputClassifier(LogisticRegression(random_state=42,solver='lbfgs', max_iter=2000))

#params with c values:
params_em= {"estimator__C": [0.0001, 0.001, 0.01, 0.1, 1, 10,100]}

#initialize GridSearchCV with scoring f1_macro:

init_grid_search_macro_f_em=GridSearchCV(log_reg_f_em,params_em,cv=5,scoring='f1_macro')

# Fit the model on X_train with scoring f1_macro:
init_grid_search_macro_f_em.fit(X_train_f_em_lex,ya_train_em)

#Validation Score with scoring f1_macro:
validation_macro_score_f_em = init_grid_search_macro_f_em.score(X_test_f_em_lex,ya_test_em) 
validation_results_best_score_f_em=init_grid_search_macro_f_em.best_score_
# Get the score from the GridSearchCV "best score" with Macro f1:
print("Macro Validation Score F1- Exclamation Feature: {:.4f}".format(validation_results_best_score_f_em))
print("Macro Test Score F1- Exclamation Feature: {:.4f}".format(validation_macro_score_f_em))

#predciting on X_test data with scoring f1_macro:
logistic_X_test_prediciton_macro_f_em=init_grid_search_macro_f_em.predict(X_test_f_em_lex)

# Calculating precision, recall and f1 Scores with average as macro parameter:
precision_macro_f_em=precision_score(ya_test_em,logistic_X_test_prediciton_macro_f_em,average='macro')  # Get scores using logistic_X_test_prediciton and y_test with the precision_score method
recall_macro_f_em = recall_score(ya_test_em,logistic_X_test_prediciton_macro_f_em,average='macro')
print("Macro_Score Precision-Exclamation Feature: {:.4f}".format(precision_macro_f_em))
print("Macro_Score Recall-Exclamation Feature: {:.4f}".format(recall_macro_f_em))
for i_ma in range(4):
    #f1_macro_i = f1_score([item[i] for item in y_test], [item[i] for item in logistic_X_test_prediciton_macro])
    f1_macro_f_em_i_ma = f1_score(ya_test_em[:,i_ma],logistic_X_test_prediciton_macro_f_em[:,i_ma])
    print(f"{header[i_ma+2]} Macro_Score F1-Exclamation Feature: {f1_macro_f_em_i_ma:.4f}")
    
print()    
    

#initialize GridSearchCV with scoring f1_micro:

init_grid_search_micro_f_em=GridSearchCV(log_reg_f_em,params_em,cv=5,scoring='f1_micro')

# Fit the model on X_train with scoring f1_micro:
init_grid_search_micro_f_em.fit(X_train_f_em_lex,ya_train_em)

#Validation Score with scoring f1_macro:
validation_micro_score_f_em = init_grid_search_micro_f_em.score(X_test_f_em_lex,ya_test_em) 
validation_results_best_score_micro_f_em=init_grid_search_micro_f_em.best_score_
# Get the score from the GridSearchCV "best score" with Macro f1:
print("Micro Validation Score F1-Exclamation Feature: {:.4f}".format(validation_results_best_score_micro_f_em))
print("Micro Test Score F1-Exclamation Feature: {:.4f}".format(validation_micro_score_f_em))

# Get the score from the GridSearchCV "best score" with Micro f1:
#print("Micro_Score Validation F1: {:.4f}".format(validation_micro_score))

#predciting on X_test data with scoring f1_micro:
logistic_X_test_prediciton_micro_f_em=init_grid_search_micro_f_em.predict(X_test_f_em_lex)

# Calculating precision, recall and f1 Scores with average as micro parameter:
precision_micro_f_em=precision_score(ya_test_em,logistic_X_test_prediciton_micro_f_em,average='micro')  # Get scores using logistic_X_test_prediciton and y_test with the precision_score method
recall_micro_f_em = recall_score(ya_test_em,logistic_X_test_prediciton_micro_f_em,average='micro')
print("Micro_Score Precision-Exclamation Feature: {:.4f}".format(precision_micro_f_em))
print("Micro_Score Recall-Exclamation Feature: {:.4f}".format(recall_micro_f_em))
for i_mi in range(4):
    #f1_micro_i = f1_score([item[i] for item in y_test], [item[i] for item in logistic_X_test_prediciton_micro])
    f1_micro_f_em_i_mi = f1_score(ya_test_em[:,i_mi],logistic_X_test_prediciton_micro_f_em[:,i_mi])
    print(f"{header[i_mi+2]} Micro_Score F1-Exclamation Feature: {f1_micro_f_em_i_mi:.4f}")

Macro Validation Score F1- Exclamation Feature: 0.3232
Macro Test Score F1- Exclamation Feature: 0.3085
Macro_Score Precision-Exclamation Feature: 0.3153
Macro_Score Recall-Exclamation Feature: 0.3078
Gold Standards- Technology Macro_Score F1-Exclamation Feature: 0.8099
Gold Standards- Ride Share Macro_Score F1-Exclamation Feature: 0.0889
Gold Standards- Food Delivery Macro_Score F1-Exclamation Feature: 0.1628
Gold Standards- Online Shopping Macro_Score F1-Exclamation Feature: 0.1724

Micro Validation Score F1-Exclamation Feature: 0.5803
Micro Test Score F1-Exclamation Feature: 0.5464
Micro_Score Precision-Exclamation Feature: 0.6158
Micro_Score Recall-Exclamation Feature: 0.4910
Gold Standards- Technology Micro_Score F1-Exclamation Feature: 0.7423
Gold Standards- Ride Share Micro_Score F1-Exclamation Feature: 0.0667
Gold Standards- Food Delivery Micro_Score F1-Exclamation Feature: 0.0000
Gold Standards- Online Shopping Micro_Score F1-Exclamation Feature: 0.0000


In [5]:
#modeling with ngram_range=(1,1) & LogisticRegression with CountVectorizer: 
#reading the necessary libraries:

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score,classification_report
from scipy.sparse import hstack 


import numpy as np
np.random.seed(42)
import random
random.seed(42)

# Generating Random multiplable classification for Logistic Regression:
#X, y = make_multilabel_classification(n_classes=4,random_state=0)
#lg_clf=LogisticRegression(solver='lbfgs', max_iter=100)
#clf = MultiOutputClassifier(lg_clf).fit(X, y)

vec=CountVectorizer(ngram_range=(1,1))
X_train_matrix =vec.fit_transform(X_txt) # This should be a matrix
#y_matrix=vec.transform(y)# This should be a matrix

#Split the data into training and test sets:
X_train,X_test,y_train,y_test=train_test_split(X_train_matrix,y,test_size=0.2)

# Now we need to convert X_train_lexicon_features and X_test_lexicon_features to numpy arrays
# "hstack" X_train_lexicon_features with X_train_w_lex
# "hstack" X_test_lexicon_features with X_test_w_lex
X_train_lexicon_farray=np.array(X_train_lexicon_features)
X_test_lexicon_farray=np.array(X_test_lexicon_features)

#converting list to array:
ya_train=np.array(y_train)
ya_test=np.array(y_test)
#print(y_test.shape[1])
#initializing logisticregression:
log_reg=MultiOutputClassifier(LogisticRegression(random_state=42,solver='lbfgs', max_iter=1000))

#params with c values:
params= {"estimator__C": [0.0001, 0.001, 0.01, 0.1, 1, 10,100]}

#initialize GridSearchCV with scoring f1_macro:

init_grid_search_macro=GridSearchCV(log_reg,params,cv=5,scoring='f1_macro')

# Fit the model on X_train with scoring f1_macro:
init_grid_search_macro.fit(X_train,ya_train)

#Validation Score with scoring f1_macro:
validation_macro_score = init_grid_search_macro.score(X_test,ya_test) 

# Get the score from the GridSearchCV "best score" with Macro f1:
print("Macro_Score Validation F1: {:.4f}".format(validation_macro_score))

#predciting on X_test data with scoring f1_macro:
logistic_X_test_prediciton_macro=init_grid_search_macro.predict(X_test)

# Calculating precision, recall and f1 Scores with average as macro parameter:
precision_macro=precision_score(ya_test,logistic_X_test_prediciton_macro,average='macro')  # Get scores using logistic_X_test_prediciton and y_test with the precision_score method
recall_macro = recall_score(ya_test,logistic_X_test_prediciton_macro,average='macro')
print("Macro_Score Precision: {:.4f}".format(precision_macro))
print("Macro_Score Recall: {:.4f}".format(recall_macro))
for i in range(4):
    f1_macro_i = f1_score(ya_test[:,i],logistic_X_test_prediciton_macro[:,i],average='macro')
    print(f"{header[i+2]} Macro_Score F1: {f1_macro_i:.4f}")
    
print()    
    

#initialize GridSearchCV with scoring f1_micro:

init_grid_search_micro=GridSearchCV(log_reg,params,cv=5,scoring='f1_micro')

# Fit the model on X_train with scoring f1_micro:
init_grid_search_micro.fit(X_train,ya_train)

#Validation Score with scoring f1_micro:
validation_micro_score = init_grid_search_micro.score(X_test, ya_test) 

# Get the score from the GridSearchCV "best score" with Micro f1:
print("Micro_Score Validation F1: {:.4f}".format(validation_micro_score))

#predciting on X_test data with scoring f1_micro:
logistic_X_test_prediciton_micro=init_grid_search_micro.predict(X_test)

# Calculating precision, recall and f1 Scores with average as micro parameter:
precision_micro=precision_score(ya_test,logistic_X_test_prediciton_micro,average='micro')  # Get scores using logistic_X_test_prediciton and y_test with the precision_score method
recall_micro = recall_score(ya_test,logistic_X_test_prediciton_micro,average='micro')
print("Micro_Score Precision: {:.4f}".format(precision_micro))
print("Micro_Score Recall: {:.4f}".format(recall_micro))
for i in range(4):
    f1_micro_i = f1_score(ya_test[:,i],logistic_X_test_prediciton_micro[:,i],average='micro')
    print(f"{header[i+2]} Micro_Score F1: {f1_micro_i:.4f}")

NameError: name 'X_train_lexicon_features' is not defined

In [6]:
#modeling with ngram_range=(1,3) & LogisticRegression with CountVectorizer: 
#reading the necessary libraries:

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score


#import numpy as np
#np.random.seed(42)
#import random
#random.seed(42)

# Generating Random multiplable classification for Logistic Regression:
#X, y = make_multilabel_classification(n_classes=4,random_state=0)
#lg_clf=LogisticRegression(solver='lbfgs', max_iter=100)
#clf = MultiOutputClassifier(lg_clf).fit(X, y)

vec=CountVectorizer(ngram_range=(1,3))
X_train_matrix =vec.fit_transform(X_txt) # This should be a matrix
#y_matrix=vec.transform(y)# This should be a matrix

#Split the data into training and test sets:
#X_train,X_test,y_train,y_test=train_test_split(X_train_matrix,y,test_size=0.2)

#converting list to array:
y_train=np.array(y_train)
y_test=np.array(y_test)

#initializing logisticregression:
log_reg=MultiOutputClassifier(LogisticRegression(random_state=42,solver='lbfgs', max_iter=1000))

#params with c values:
params= {"estimator__C": [0.0001, 0.001, 0.001, 0.01, 0.1, 1, 10,100]}

#initialize GridSearchCV with scoring f1_macro:

init_grid_search_macro=GridSearchCV(log_reg,params,cv=5,scoring='f1_macro')

# Fit the model on X_train with scoring f1_macro:
init_grid_search_macro.fit(X_train,y_train)

#Validation Score with scoring f1_macro:
validation_macro_score = init_grid_search_macro.score(X_test, y_test) 

# Get the score from the GridSearchCV "best score" with Macro f1:
print("Macro_Score Validation F1: {:.4f}".format(validation_macro_score))

#predciting on X_test data with scoring f1_macro:
logistic_X_test_prediciton_macro=init_grid_search_macro.predict(X_test)

# Calculating precision, recall and f1 Scores with average as macro parameter:
precision_macro=precision_score(y_test,logistic_X_test_prediciton_macro,average='macro')  # Get scores using logistic_X_test_prediciton and y_test with the precision_score method
recall_macro = recall_score(y_test,logistic_X_test_prediciton_macro,average='macro')
print("Macro_Score Precision: {:.4f}".format(precision_macro))
print("Macro_Score Recall: {:.4f}".format(recall_macro))
for i in range(4):
    f1_macro_i = f1_score(y_test[:,i],logistic_X_test_prediciton_macro[:,i],average='macro')
    print(f"{header[i+2]} Macro_Score F1: {f1_macro_i:.4f}")
    
    
    

#initialize GridSearchCV with scoring f1_micro:

init_grid_search_micro=GridSearchCV(log_reg,params,cv=5,scoring='f1_micro')

# Fit the model on X_train with scoring f1_micro:
init_grid_search_micro.fit(X_train,y_train)

#Validation Score with scoring f1_micro:
validation_micro_score = init_grid_search_micro.score(X_test, y_test) 

# Get the score from the GridSearchCV "best score" with Micro f1:
print("Micro_Score Validation F1: {:.4f}".format(validation_micro_score))

#predciting on X_test data with scoring f1_micro:
logistic_X_test_prediciton_micro=init_grid_search_micro.predict(X_test)

# Calculating precision, recall and f1 Scores with average as micro parameter:
precision_micro=precision_score(y_test,logistic_X_test_prediciton_micro,average='micro')  # Get scores using logistic_X_test_prediciton and y_test with the precision_score method
recall_micro = recall_score(y_test,logistic_X_test_prediciton_micro,average='micro')
print("Micro_Score Precision: {:.4f}".format(precision_micro))
print("Micro_Score Recall: {:.4f}".format(recall_micro))
for i in range(4):
    f1_micro_i = f1_score(y_test[:,i],logistic_X_test_prediciton_micro[:,i],average='micro')
    print(f"{header[i+2]} Micro_Score F1: {f1_micro_i:.4f}")    

Macro_Score Validation F1: 0.3034
Macro_Score Precision: 0.3334
Macro_Score Recall: 0.2923
Gold Standards- Technology Macro_Score F1: 0.7765
Gold Standards- Ride Share Macro_Score F1: 0.5211
Gold Standards- Food Delivery Macro_Score F1: 0.5346
Gold Standards- Online Shopping Macro_Score F1: 0.5179
Micro_Score Validation F1: 0.6494
Micro_Score Precision: 0.7692
Micro_Score Recall: 0.5618
Gold Standards- Technology Micro_Score F1: 0.7900
Gold Standards- Ride Share Micro_Score F1: 0.8750
Gold Standards- Food Delivery Micro_Score F1: 0.9350
Gold Standards- Online Shopping Micro_Score F1: 0.8600


In [5]:
#modeling with ngram_range=(1,5) & LogisticRegression with CountVectorizer: 
#reading the necessary libraries:

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score


#import numpy as np
#np.random.seed(42)
#import random
#random.seed(42)

# Generating Random multiplable classification for Logistic Regression:
#X, y = make_multilabel_classification(n_classes=4,random_state=0)
#lg_clf=LogisticRegression(solver='lbfgs', max_iter=100)
#clf = MultiOutputClassifier(lg_clf).fit(X, y)

vec=CountVectorizer(ngram_range=(1,5))
X_train_matrix =vec.fit_transform(X_txt) # This should be a matrix
#y_matrix=vec.transform(y)# This should be a matrix

#Split the data into training and test sets:
#X_train,X_test,y_train,y_test=train_test_split(X_train_matrix,y,test_size=0.2)

#converting list to array:
y_train=np.array(y_train)
y_test=np.array(y_test)

#initializing logisticregression:
log_reg=MultiOutputClassifier(LogisticRegression(random_state=42,solver='lbfgs', max_iter=1000))

#params with c values:
params= {"estimator__C": [0.0001, 0.001, 0.001, 0.01, 0.1, 1, 10,100]}

#initialize GridSearchCV with scoring f1_macro:

init_grid_search_macro=GridSearchCV(log_reg,params,cv=5,scoring='f1_macro')

# Fit the model on X_train with scoring f1_macro:
init_grid_search_macro.fit(X_train,y_train)

#Validation Score with scoring f1_macro:
validation_macro_score = init_grid_search_macro.score(X_test, y_test) 

# Get the score from the GridSearchCV "best score" with Macro f1:
print("Macro_Score Validation F1: {:.4f}".format(validation_macro_score))

#predciting on X_test data with scoring f1_macro:
logistic_X_test_prediciton_macro=init_grid_search_macro.predict(X_test)

# Calculating precision, recall and f1 Scores with average as macro parameter:
precision_macro=precision_score(y_test,logistic_X_test_prediciton_macro,average='macro')  # Get scores using logistic_X_test_prediciton and y_test with the precision_score method
recall_macro = recall_score(y_test,logistic_X_test_prediciton_macro,average='macro')
print("Macro_Score Precision: {:.4f}".format(precision_macro))
print("Macro_Score Recall: {:.4f}".format(recall_macro))
for i in range(4):
    f1_macro_i = f1_score(y_test[:,i],logistic_X_test_prediciton_macro[:,i],average='macro')
    print(f"{header[i+2]} Macro_Score F1: {f1_macro_i:.4f}")
    
    
    

#initialize GridSearchCV with scoring f1_micro:

init_grid_search_micro=GridSearchCV(log_reg,params,cv=5,scoring='f1_micro')

# Fit the model on X_train with scoring f1_micro:
init_grid_search_micro.fit(X_train,y_train)

#Validation Score with scoring f1_micro:
validation_micro_score = init_grid_search_micro.score(X_test, y_test) 

# Get the score from the GridSearchCV "best score" with Micro f1:
print("Micro_Score Validation F1: {:.4f}".format(validation_micro_score))

#predciting on X_test data with scoring f1_micro:
logistic_X_test_prediciton_micro=init_grid_search_micro.predict(X_test)

# Calculating precision, recall and f1 Scores with average as micro parameter:
precision_micro=precision_score(y_test,logistic_X_test_prediciton_micro,average='micro')  # Get scores using logistic_X_test_prediciton and y_test with the precision_score method
recall_micro = recall_score(y_test,logistic_X_test_prediciton_micro,average='micro')
print("Micro_Score Precision: {:.4f}".format(precision_micro))
print("Micro_Score Recall: {:.4f}".format(recall_micro))
for i in range(4):
    f1_micro_i = f1_score(y_test[:,i],logistic_X_test_prediciton_micro[:,i],average='micro')
    print(f"{header[i+2]} Micro_Score F1: {f1_micro_i:.4f}")    

Macro_Score Validation F1: 0.3034
Macro_Score Precision: 0.3334
Macro_Score Recall: 0.2923
Gold Standards- Technology Macro_Score F1: 0.7765
Gold Standards- Ride Share Macro_Score F1: 0.5211
Gold Standards- Food Delivery Macro_Score F1: 0.5346
Gold Standards- Online Shopping Macro_Score F1: 0.5179
Micro_Score Validation F1: 0.6494
Micro_Score Precision: 0.7692
Micro_Score Recall: 0.5618
Gold Standards- Technology Micro_Score F1: 0.7900
Gold Standards- Ride Share Micro_Score F1: 0.8750
Gold Standards- Food Delivery Micro_Score F1: 0.9350
Gold Standards- Online Shopping Micro_Score F1: 0.8600


In [8]:
#modeling with ngram_range=(2,2) & LogisticRegression with CountVectorizer: 
#reading the necessary libraries:

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer


#import numpy as np
#np.random.seed(42)
#import random
#random.seed(42)

# Generating Random multiplable classification for Logistic Regression:
#X, y = make_multilabel_classification(n_classes=4,random_state=0)
#lg_clf=LogisticRegression(solver='lbfgs', max_iter=100)
#clf = MultiOutputClassifier(lg_clf).fit(X, y)

vec=TfidfVectorizer(ngram_range=(2,2))
X_train_matrix =vec.fit_transform(X_txt) # This should be a matrix
#y_matrix=vec.transform(y)# This should be a matrix

#Split the data into training and test sets:
#X_train,X_test,y_train,y_test=train_test_split(X_train_matrix,y,test_size=0.2)

#converting list to array:
y_train=np.array(y_train)
y_test=np.array(y_test)

#initializing logisticregression:
log_reg=MultiOutputClassifier(LogisticRegression(random_state=42,solver='lbfgs', max_iter=1000))

#params with c values:
params= {"estimator__C": [0.0001, 0.001, 0.001, 0.01, 0.1, 1, 10,100]}

#initialize GridSearchCV with scoring f1_macro:

init_grid_search_macro=GridSearchCV(log_reg,params,cv=5,scoring='f1_macro')

# Fit the model on X_train with scoring f1_macro:
init_grid_search_macro.fit(X_train,y_train)

#Validation Score with scoring f1_macro:
validation_macro_score = init_grid_search_macro.score(X_test, y_test) 

# Get the score from the GridSearchCV "best score" with Macro f1:
print("Macro_Score Validation F1: {:.4f}".format(validation_macro_score))

#predciting on X_test data with scoring f1_macro:
logistic_X_test_prediciton_macro=init_grid_search_macro.predict(X_test)

# Calculating precision, recall and f1 Scores with average as macro parameter:
precision_macro=precision_score(y_test,logistic_X_test_prediciton_macro,average='macro')  # Get scores using logistic_X_test_prediciton and y_test with the precision_score method
recall_macro = recall_score(y_test,logistic_X_test_prediciton_macro,average='macro')
print("Macro_Score Precision: {:.4f}".format(precision_macro))
print("Macro_Score Recall: {:.4f}".format(recall_macro))
for i in range(4):
    f1_macro_i = f1_score(y_test[:,i],logistic_X_test_prediciton_macro[:,i],average='macro')
    print(f"{header[i+2]} Macro_Score F1: {f1_macro_i:.4f}")
    
    
    

#initialize GridSearchCV with scoring f1_micro:

init_grid_search_micro=GridSearchCV(log_reg,params,cv=5,scoring='f1_micro')

# Fit the model on X_train with scoring f1_micro:
init_grid_search_micro.fit(X_train,y_train)

#Validation Score with scoring f1_micro:
validation_micro_score = init_grid_search_micro.score(X_test, y_test) 

# Get the score from the GridSearchCV "best score" with Micro f1:
print("Micro_Score Validation F1: {:.4f}".format(validation_micro_score))

#predciting on X_test data with scoring f1_micro:
logistic_X_test_prediciton_micro=init_grid_search_micro.predict(X_test)

# Calculating precision, recall and f1 Scores with average as micro parameter:
precision_micro=precision_score(y_test,logistic_X_test_prediciton_micro,average='micro')  # Get scores using logistic_X_test_prediciton and y_test with the precision_score method
recall_micro = recall_score(y_test,logistic_X_test_prediciton_micro,average='micro')
print("Micro_Score Precision: {:.4f}".format(precision_micro))
print("Micro_Score Recall: {:.4f}".format(recall_micro))
for i in range(4):
    f1_micro_i = f1_score(y_test[:,i],logistic_X_test_prediciton_micro[:,i],average='micro')
    print(f"{header[i+2]} Micro_Score F1: {f1_micro_i:.4f}")    

Macro_Score Validation F1: 0.3034
Macro_Score Precision: 0.3334
Macro_Score Recall: 0.2923
Gold Standards- Technology Macro_Score F1: 0.7765
Gold Standards- Ride Share Macro_Score F1: 0.5211
Gold Standards- Food Delivery Macro_Score F1: 0.5346
Gold Standards- Online Shopping Macro_Score F1: 0.5179
Micro_Score Validation F1: 0.6494
Micro_Score Precision: 0.7692
Micro_Score Recall: 0.5618
Gold Standards- Technology Micro_Score F1: 0.7900
Gold Standards- Ride Share Micro_Score F1: 0.8750
Gold Standards- Food Delivery Micro_Score F1: 0.9350
Gold Standards- Online Shopping Micro_Score F1: 0.8600
